In [ ]:
import tensorflow as tf
import numpy as np
import sys
import h5py
import random
import math
from math import sqrt
import argparse

#thanks to https://gist.githubusercontent.com/kukuruza/03731dc494603ceab0c5/raw/3d708320145df0a962cfadb95b3f716b623994e0/gist_cifar10_train.py
def put_kernels_on_grid (kernel, grid_Y, grid_X, pad = 1):

    '''Visualize conv. features as an image (mostly for the 1st layer).
    Place kernel into a grid, with some paddings between adjacent filters.

    Args:
      kernel:            tensor of shape [Y, X, NumChannels, NumKernels]
      (grid_Y, grid_X):  shape of the grid. Require: NumKernels == grid_Y * grid_X
                           User is responsible of how to break into two multiples.
      pad:               number of black pixels around each filter (between them)

    Return:
      Tensor of shape [(Y+2*pad)*grid_Y, (X+2*pad)*grid_X, NumChannels, 1].
    '''

    x_min = tf.reduce_min(kernel)
    x_max = tf.reduce_max(kernel)

    kernel1 = (kernel - x_min) / (x_max - x_min)

    # pad X and Y
    x1 = tf.pad(kernel1, tf.constant( [[pad,pad],[pad, pad],[0,0],[0,0]] ), mode = 'CONSTANT')

    # X and Y dimensions, w.r.t. padding
    Y = kernel1.get_shape()[0] + 2 * pad
    X = kernel1.get_shape()[1] + 2 * pad

    channels = kernel1.get_shape()[2]

    # put NumKernels to the 1st dimension
    x2 = tf.transpose(x1, (3, 0, 1, 2))
    # organize grid on Y axis
    x3 = tf.reshape(x2, tf.stack([grid_X, Y * grid_Y, X, channels])) #3

    # switch X and Y axes
    x4 = tf.transpose(x3, (0, 2, 1, 3))
    # organize grid on X axis
    x5 = tf.reshape(x4, tf.stack([1, X * grid_X, Y * grid_Y, channels])) #3

    # back to normal order (not combining with the next step for clarity)
    x6 = tf.transpose(x5, (2, 1, 3, 0))

    # to tf.image_summary order [batch_size, height, width, channels],
    #   where in this case batch_size == 1
    x7 = tf.transpose(x6, (3, 0, 1, 2))

    # scale to [0, 255] and convert to uint8
    return tf.image.convert_image_dtype(x7, dtype = tf.uint8)

def variable_summaries(var):
    """Attach a lot of summaries to a Tensor (for TensorBoard visualization)."""
    with tf.name_scope('summaries'):
        mean = tf.reduce_mean(var)
        meansum = tf.summary.scalar('mean', mean)
        with tf.name_scope('stddev'):
              stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
        stdv = tf.summary.scalar('stddev', stddev)
        maxsum = tf.summary.scalar('max', tf.reduce_max(var))
        minsum = tf.summary.scalar('min', tf.reduce_min(var))
        hist = tf.summary.histogram('histogram', var)
        
        return tf.summary.merge([meansum,stdv,maxsum,minsum,hist])


def main(Learning_Rate,multiplier):

    trainfile = "/om/user/ribeirop/audiosetDL/balanced_stripped/balanced_train_segments_Coch.hdf5"
    train_averagefile = "/om/user/ribeirop/audiosetDL/balanced_stripped/balanced_train_segments_average.npy"
    testfile = trainfile# "/om/user/ribeirop/audiosetDL/eval_stripped/eval_segments_Coch.hdf5"
    test_averagefile = train_averagefile# "/om/user/ribeirop/audiosetDL/eval_stripped/eval_segments_average.npy"
    trainset = h5py.File(trainfile)
    testset = h5py.File(testfile)
    
    train_ave_coch = np.load(train_averagefile)
    test_ave_coch = np.load(test_averagefile)
    
    
    
    Num_Epochs = 100
    
    COCHLEAGRAM_LENGTH = int(342000/1) # full is 342000
    
    limit = 128*multiplier
    
    trainsize = limit#trainset.attrs.get("size")
    testsize = limit#testset.attrs.get("size")
    
    b = 0
    batchsize = 1
    
    final_filter = 512
    
    TensorBoard_Folder = './TB/VGGtrain_n{0}_lr{1}'.format(limit,Learning_Rate)
    
    full_length = final_filter * math.ceil(COCHLEAGRAM_LENGTH/171/2/2/2/2/2)* math.ceil(171/2/2/2/2/2)
    
    nets = {}
    
    print("starting")
    
    with tf.Graph().as_default():
        with tf.name_scope('input'):
            nets['input_to_net'] = tf.placeholder(tf.float32, [None,COCHLEAGRAM_LENGTH], name='input_to_net')
            nets['actual_labels'] = tf.placeholder(tf.float32, [None, 527], name='actual_labels')
            nets['label_indeces'] = tf.placeholder(tf.int32, [None, 15], name='label_indeces')
            keep_prob = tf.placeholder(tf.float32)
            
            nets['accuracy'] = tf.placeholder(tf.float32, (), name='acc')
            nets['accsum'] = tf.summary.scalar("accuracy",nets['accuracy'])
            
            
            nets['reshapedCoch'] = tf.reshape(nets['input_to_net'],[-1,171,int(COCHLEAGRAM_LENGTH/171),1], name='reshape_input')

        #sess.run(tf.global_variables_initializer())
        #thing = sess.run(reshapedCoch,feed_dict={input_to_net:trainset["/coch"][0:2],actual_labels:trainset["/labels"][0:2]})

        with tf.variable_scope('conv1') as scope:
            #Conv_1_1 and save the graphs
            tf.get_variable_scope().reuse_variables()
            conv1_1_Weights = weight_variable([3,3,1,64])
            
            layer1 = variable_summaries(conv1_1_Weights)
            
            grid = put_kernels_on_grid(conv1_1_Weights,8,8)
            nets['conv1_weight_image'] = tf.summary.image('conv1/kernels', grid, max_outputs=3)       
            nets['conv1_1'] = conv2d(nets['reshapedCoch'], Weights = conv1_1_Weights, bias = bias_variable([64]), strides=1, name='conv1_1')
                
            #conv_1_2         
            nets['conv1_2'] = conv2d(nets['conv1_1'], Weights = weight_variable([3,3,64,64]), bias = bias_variable([64]), strides=1, name='conv1_2')       
            nets['maxpool1'] = maxpool2x2(nets['conv1_2'],k=2 , name='maxpool1')
        
        with tf.name_scope('conv2'):
            nets['conv2_1'] = conv2d(nets['maxpool1'], Weights = weight_variable([3,3,64,128]), bias = bias_variable([128]), strides=1, name='conv2_1')        
            nets['conv2_2'] = conv2d(nets['conv2_1'], Weights = weight_variable([3,3,128,128]), bias = bias_variable([128]), strides=1, name='conv2_2')                  
            nets['maxpool2'] = maxpool2x2(nets['conv2_2'],k=2 , name='maxpool2')
            
            
        with tf.name_scope('conv3'):
            nets['conv3_1'] = conv2d(nets['maxpool2'], Weights = weight_variable([3,3,128,256]), bias = bias_variable([256]), strides=1, name='conv3_1')            
            nets['conv3_2'] = conv2d(nets['conv3_1'], Weights = weight_variable([3,3,256,256]), bias = bias_variable([256]), strides=1, name='conv3_2')       
            nets['conv3_3'] = conv2d(nets['conv3_2'], Weights = weight_variable([3,3,256,256]), bias = bias_variable([256]), strides=1, name='conv3_3')           
            nets['conv3_4'] = conv2d(nets['conv3_3'], Weights = weight_variable([3,3,256,256]), bias = bias_variable([256]), strides=1, name='conv3_4')       
        
            nets['maxpool3'] = maxpool2x2(nets['conv3_4'],k=2 , name='maxpool3')
            
        with tf.name_scope('conv4'):
            nets['conv4_1'] = conv2d(nets['maxpool3'], Weights = weight_variable([3,3,256,512]), bias = bias_variable([512]), strides=1, name='conv4_1')            
            nets['conv4_2'] = conv2d(nets['conv4_1'], Weights = weight_variable([3,3,512,512]), bias = bias_variable([512]), strides=1, name='conv4_2')       
            nets['conv4_3'] = conv2d(nets['conv4_2'], Weights = weight_variable([3,3,512,512]), bias = bias_variable([512]), strides=1, name='conv4_3')           
            nets['conv4_4'] = conv2d(nets['conv4_3'], Weights = weight_variable([3,3,512,512]), bias = bias_variable([512]), strides=1, name='conv4_4')       
        
            nets['maxpool4'] = maxpool2x2(nets['conv4_4'],k=2 , name='maxpool4')
            
            
        with tf.name_scope('conv5'):
            nets['conv5_1'] = conv2d(nets['maxpool4'], Weights = weight_variable([3,3,512,512]), bias = bias_variable([512]), strides=1, name='conv5_1')            
            nets['conv5_2'] = conv2d(nets['conv5_1'], Weights = weight_variable([3,3,512,512]), bias = bias_variable([512]), strides=1, name='conv5_2')       
            nets['conv5_3'] = conv2d(nets['conv5_2'], Weights = weight_variable([3,3,512,512]), bias = bias_variable([512]), strides=1, name='conv5_3')           
            nets['conv5_4'] = conv2d(nets['conv5_3'], Weights = weight_variable([3,3,512,512]), bias = bias_variable([512]), strides=1, name='conv5_4')       
        
            nets['maxpool5'] = maxpool2x2(nets['conv5_4'],k=2 , name='maxpool5')
            
            nets['flattened'] = tf.reshape(nets['maxpool5'], [-1, full_length])
        
        with tf.name_scope('fc_1'):
            W_fc1 = weight_variable([full_length, 4096]) # 4,959,232
            b_fc1 = bias_variable([4096])
            nets['fc_1'] = tf.nn.relu(tf.matmul(nets['flattened'], W_fc1) + b_fc1)
            nets['h_fc1_drop'] = tf.nn.dropout(nets['fc_1'], keep_prob)
        
        with tf.name_scope('fc_2'):
            W_fc2 = weight_variable([4096, 4096]) # 4,959,232
            b_fc2 = bias_variable([4096])
            nets['fc_2'] = tf.nn.relu(tf.matmul(nets['h_fc1_drop'], W_fc2) + b_fc2)
            nets['h_fc2_drop'] = tf.nn.dropout(nets['fc_2'], keep_prob)
            
        with tf.name_scope('fc_3'):
            W_fc3 = weight_variable([4096, 527],name='W_fc3') # 4,959,232
            b_fc3 = bias_variable([527],name = 'b_fc3')
            nets['predicted_labels'] = tf.nn.relu(tf.matmul(nets['h_fc2_drop'], W_fc3) + b_fc3)

            
        with tf.variable_scope("eval"):
            nets['cross_entroy'] = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels =nets['actual_labels'],logits = nets['predicted_labels'] ))
            nets['train_step'] = tf.train.AdamOptimizer(Learning_Rate).minimize(nets['cross_entroy'])

            nets['cross_entropy_summary'] = tf.summary.scalar("cross_entropy_summary",nets['cross_entroy'])
            
            #num = np.sum(nets['actual_labelsi'])
            #nets['accuracy'] = tf.metrics.mean(tf.nn.in_top_k(nets['predicted_labels'],nets['actual_labelsi'],2))

            #calculating accuracys

            _,nets['indeces'] = tf.nn.top_k(nets['predicted_labels'],15)
            prediction_sums = tf.summary.histogram('predicted_labels_histogram',nets['indeces'])
            nets['numCorrect'] = tf.shape(tf.sets.set_intersection(nets['indeces'],nets['label_indeces'],False).values)[0]
        
        merged = tf.summary.merge_all()
        print("beginnning session")
        with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())
            train_writer = tf.summary.FileWriter(TensorBoard_Folder, sess.graph)
            
            for epoch in range(Num_Epochs):
                print("epoch: ",epoch)
                print("training")
                
                #Train a full epoch
                startindex = 0
                endindex = startindex+batchsize
                while(endindex <= trainsize):
                    print(startindex)
                    _,cross_entropySummary,l1Sums = sess.run([nets['train_step'],nets['cross_entropy_summary'],layer1],feed_dict={nets['input_to_net']:trainset["/coch"][startindex:endindex][:,0:COCHLEAGRAM_LENGTH]-train_ave_coch[0:COCHLEAGRAM_LENGTH],nets['actual_labels']:trainset["/labels"][startindex:endindex],keep_prob:.5})
                    train_writer.add_summary(cross_entropySummary, epoch)
                    train_writer.add_summary(l1Sums, epoch)
                    
                    startindex = endindex
                    endindex = startindex+batchsize
                    
                
                print("eval")
                #Evaluate the full epoch
                startindex = 0
                endindex = startindex+batchsize
                total_count = 0
                correct = 0
                while(endindex <= testsize):
                    print(startindex)
                    trainBatchLabels = testset["/labels"][startindex:endindex]
                    indeces_list = []
                    for row in trainBatchLabels:
                        indeces= np.where(row==1)[0]
                        total_count = total_count + len(indeces)
                        indeces_list.append(np.pad(indeces,[0,15-len(indeces)],mode = 'constant',constant_values=-1))
                        
                    
                    addme,pred_sum = sess.run([nets['numCorrect'],prediction_sums],feed_dict={nets['input_to_net']:testset["/coch"][startindex:endindex][:,0:COCHLEAGRAM_LENGTH]-test_ave_coch[0:COCHLEAGRAM_LENGTH],nets['actual_labels']:testset["/labels"][startindex:endindex],nets['label_indeces']:indeces_list,keep_prob:1})
                    train_writer.add_summary(pred_sum, epoch + startindex/testsize)
                    correct = correct + addme
                    
                    startindex = endindex
                    endindex = startindex+batchsize
                
                
                summary = sess.run(nets['accsum'],feed_dict={nets['accuracy']:correct/total_count})
                train_writer.add_summary(summary, epoch)
                
                image_conv1 = sess.run(nets['conv1_weight_image'])
                train_writer.add_summary(image_conv1, epoch)
    

                
                startindex = random.randint(0,testsize-batchsize)
                endindex = startindex+batchsize
                potato = sess.run(nets['indeces'],feed_dict={nets['input_to_net']:testset["/coch"][startindex:endindex][:,0:COCHLEAGRAM_LENGTH]-test_ave_coch[0:COCHLEAGRAM_LENGTH],nets['actual_labels']:testset["/labels"][startindex:endindex],nets['label_indeces']:indeces_list,keep_prob:1})
                #print()
                #print("Epoch {0}, accuracy {1}".format(epoch,correct/total_count))
                #print(start)
                #print(end)
                #print(total_count)
                #print(correct)
                #print("actual: ", indeces_list)
                #print("predicted :", potato)
                #print()
                #f= open("output3.csv","a")
                #f.write("loop {0}, accuracy {1}\n".format(i,correct/total_count))
                #f.close()''
                
    print("done")
            
           
    
def weight_variable(shape, name = None):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial, name=name)

def bias_variable(shape,name = None):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial, name=name)

#builds the components
def conv2d(inputtensor, Weights, bias, strides=1,name = None):
    # Conv2D wrapper, with bias and relu activation
    x = tf.nn.conv2d(inputtensor, Weights, strides=[1, strides, strides, 1], padding='SAME',name=name)
    x = tf.nn.bias_add(x, bias)
    return tf.nn.relu(x)

def maxpool2x2(x, k=2, name = None):
    # MaxPool2D wrapper
    return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1],
                          padding='SAME',name=name)
    
if __name__ == '__main__':
    debug = True #if true, then use the default csv file to read from. 
    if debug:
        learning_rate = 1e-4
        multiplier = 1
        main(learning_rate,multiplier)
    else:
        
        parser = argparse.ArgumentParser()
        parser.add_argument('-a', '--learning_rate')
        parser.add_argument('-b', '--multiplier')

        args = vars(parser.parse_args())

        learning_rate = float(args["learning_rate"])
        multiplier = int(args["multiplier"])
        
        print(learning_rate)
        print(multiplier)

        main(learning_rate,multiplier)
        

starting
beginnning session


In [ ]:
print("hi")